# The Prime Number Theorem

In [ ]:
#@title Final program found by AlphaEvolve
import itertools
import logging
import time
from scipy import integrate
import numba
import numpy as np
from scipy.integrate import IntegrationWarning
from scipy import optimize
import warnings
import random
from sympy import mobius
import cma

def search_for_best_partial_function() -> float:
  """Function to search for the best partial function.

  Returns a dictionary with positive integer keys, and real values"""
  best_partial_function = {}
  best_score = -1.0
  time_start = time.time()

  max_keys = 50  # Increased Maximum number of keys to optimize simultaneously
  best_partial_function = {1: 3.47155351603266, 2: -1.0, 3: -1.0, 5: -1.0, 30: 1.0}
  best_score = evaluate_partial_function(best_partial_function,1000)



  for num_keys in range(5, max_keys + 1):
    prime_base = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]

    def generate_keys(current_keys, max_keys, best_score, prime_base):
      new_keys = set(current_keys)

      # Expand based on Mobius function
      for k in current_keys:
        mobius_val = mobius(k)
        if mobius_val != 0:
          for p in prime_base:
            new_key = k * p
            if new_key <= max_keys * 12 and new_key not in new_keys:
                new_keys.add(new_key)

          # Add divisors
          for i in range(2, int(np.sqrt(k)) + 1):  # optimized divisor search
              if k % i == 0:
                  if i <= max_keys * 12 and i not in new_keys:
                        new_keys.add(i)
                  if k // i <= max_keys * 12 and k // i not in new_keys:
                        new_keys.add(k // i)

      # Add some random keys based on prime combinations
      for _ in range(int(len(current_keys) * 0.2)): # adding 20% new keys
          num_primes = random.randint(1, 4)
          selected_primes = np.random.choice(prime_base, num_primes, replace=False)
          new_key = np.prod(selected_primes)
          if new_key <= max_keys * 12:
              new_keys.add(new_key)

      return list(new_keys)
    keys = generate_keys(list(best_partial_function.keys()), num_keys, best_score, prime_base)


    mobius_values = {k: mobius(k) for k in keys}
    initial_values = [val + 0.1 * mobius_values.get(k, 0) for k, val in zip(keys, list(np.random.uniform(-1, 1, size=len(keys))))]


    initial_values = list(np.random.uniform(-1, 1, size=len(keys) * 3)) #  three coefficients per key

    use_harmonic_basis = random.choice([True, False])  # Randomly decide to use harmonic basis

    def generate_harmonic_bases(keys):
        harmonic_bases = {}
        for k in keys:
            num_frequencies = random.randint(1, 4)
            frequencies = np.random.choice(range(1, 5), num_frequencies, replace=False) # use some frequencies as indices
            harmonic_bases[k] = [(f/k) for f in frequencies]  # Store these for later construction
        return harmonic_bases



    harmonic_bases = generate_harmonic_bases(keys)



    def eval_func(x, num_samples):
        curr_partial_function = best_partial_function.copy()
        if use_harmonic_basis:

          for idx, k in enumerate(keys):
            value = 0
            for i, frequency in enumerate(harmonic_bases[k]):
                value += (x[idx*3 +0] * np.sin(2 * np.pi * frequency) +
                   x[idx*3 +1] * np.cos(2 * np.pi * frequency)+
                   x[idx*3 + 2] * np.cos(4 * np.pi * frequency))
            curr_partial_function[int(k)] =value
        else:
            for i, val in zip(keys, x):
                curr_partial_function[int(i)] = val  # ensure keys are integers

        return -evaluate_partial_function(curr_partial_function, num_samples)


    bounds = [(-10, 10)] * len(keys) * 3 if use_harmonic_basis else [(-10, 10)] * len(keys)
    best_found = False

    es = cma.CMAEvolutionStrategy(initial_values, 0.5 / np.sqrt(num_keys), {"bounds": [[-10] * (len(keys) *3 if use_harmonic_basis else len(keys)), [10] *  (len(keys) * 3 if use_harmonic_basis else len(keys))], 'verbose': -9, 'tolfun': 1e-4})
    es.optimize(lambda x: eval_func(x, 1000 + 500 * (num_keys - 1)))


    if -es.result.fbest > best_score:
        best_score = -es.result.fbest
        best_found = True
        if use_harmonic_basis:
            for i, k in enumerate(keys):
              value = 0
              for j, frequency in enumerate(harmonic_bases[k]):
                  value += (es.result.xbest[i*3 + 0] * np.sin(2 * np.pi * frequency)+
                       es.result.xbest[i*3+1] * np.cos(2 * np.pi * frequency) +
                      es.result.xbest[i * 3 + 2] * np.cos(4 * np.pi * frequency))

              best_partial_function[int(k)] = value

        else:
          for i, val in zip(keys, es.result.xbest):
            best_partial_function[int(i)] = val
        print(f"Improved score with CMA-ES: {best_score:.4f} with keys: {keys}, harmonic basis:{use_harmonic_basis}")





    if best_found:
        if time.time() - time_start > 98:
            break



  return best_partial_function

**Prompt used**

Act as an expert software developer and number theory specialist specializing in creating partial functions from a subset of the positive integers to the reals with certain properties. Your task is to generate a partial function, that maximizes a hidden evaluation function called evaluate_partial_function(partial_function, num_samples) that has something to do with number theory.

Keep in mind that the optimal function will have to be defined on way more than just a handful positive integers. All function values will always be clipped to [-10, 10] during the evaluation. Once you are getting closer to the optimal score of 1.0, try to see if you can guess the function you are constructing, based on the partial functions that have performed well so far.

Your program will be evaluated based on the best function it can produce within 100 seconds. If your function doesn't return anything within 100 seconds, it will be automatically disqualified. You will also be shown previous programs that did well, see below. You can also see what function they ended up with in the end. Feel free to start your search with this function, if you so wish.

You may call the evaluate_partial_function(partial_function, num_samples) yourself to check the score of any function you are considering. The num_samples parameter determines the accuracy of this evaluation. The higher number the more accurate it will be, but of course evaluation will be slower. After you return a function, it will be scored by calling this evaluate_partial_function(partial_function, num_samples) function with num_samples = 10_000_000.

In [ ]:
#@title Initial program used

import itertools
import logging
import time
from scipy import integrate
import numba
import numpy as np
from scipy.integrate import IntegrationWarning
from scipy import optimize
import warnings

def search_for_best_partial_function() -> float:
  """Function to search for the best partial function.

  Returns a dictionary with positive integer keys, and real values"""
  curr_partial_function = {1: 0.923, 2: -2.34, 5: -0.11, 6: -0.22, 7: -1.09, 8: 0.2}
  best_partial_function = curr_partial_function.copy()
  best_score = evaluate_partial_function(curr_partial_function, 1_000_000)
  time_start = time.time()
  loop_count = 0
  while time.time() - time_start < 100:
    loop_count += 1
    for k in curr_partial_function:
      curr_partial_function[k] = np.clip((curr_partial_function[k] + 0.046254*np.sqrt(loop_count)) % 1 - 0.5, -10, 10)
      score = evaluate_partial_function(curr_partial_function, 10)
      if score > best_score:
        score = evaluate_partial_function(curr_partial_function, 100_000)
        if score > best_score:
          best_score = score
          best_partial_function = curr_partial_function.copy()
          print(score, curr_partial_function)
  return best_partial_function

In [ ]:
#@title Evaluation function


@numba.njit(fastmath=True)
def evaluate_partial_function_vectorized_numba(
    partial_function_keys, partial_function_values, num_samples
):
  """Numba-optimized vectorized function."""

  max_key_in_function = np.max(partial_function_keys)
  upper_bound_for_sampling = 10 * max_key_in_function

  for _ in range(num_samples):
    x_sample = np.random.uniform(1, upper_bound_for_sampling)
    x_sum = np.sum(
        partial_function_values * (x_sample // partial_function_keys)
    )

    if x_sum > 1.0001:
      return -np.inf

  a_value = - np.sum(
      partial_function_values
      * np.log(partial_function_keys)
      / partial_function_keys
  )
  return a_value


def evaluate_partial_function(
    partial_function: Mapping[int, float], num_samples
):
  # Create a new dictionary to avoid modifying the original
  clipped_partial_function = {}
  for k, v in partial_function.items():
    clipped_partial_function[k] = np.clip(v, -10, 10)

  total_sum = 0
  for k in clipped_partial_function:  # Use the clipped dictionary here
    total_sum += clipped_partial_function[k] / k
  if 1 in clipped_partial_function:  # Use the clipped dictionary here
    clipped_partial_function[1] -= total_sum  # Use the clipped dictionary here
  else:
    clipped_partial_function[1] = -total_sum  # Use the clipped dictionary here

  keys_np = np.array(
      list(clipped_partial_function.keys())
  )  # Use the clipped dictionary here
  values_np = np.array(
      list(clipped_partial_function.values())
  )  # Use the clipped dictionary here
  return evaluate_partial_function_vectorized_numba(
      keys_np, values_np, num_samples
  )